# Welcome to conu demo! 

You need to install following packages to be able to run docker examples: `docker` and `conu`, for kubernetes and OpenShift examples I recommend to run local cluster using `minishift`.

In [1]:
from IPython.display import display, HTML, Pretty

# Docker example

In [2]:
from conu import DockerBackend

In [3]:
backend = DockerBackend()

10:32:55.788 backend.py        INFO   conu has initiated, welcome to the party!
10:32:56.644 __init__.py       INFO   docker environment info: 'Client: Docker Engine - Community\n Version:           18.09.1\n API version:       1.39\n Go version:        go1.10.6\n Git commit:        4c52b90\n Built:             Wed Jan  9 19:33:12 2019\n OS/Arch:           darwin/amd64\n Experimental:      false\n\nServer: Docker Engine - Community\n Engine:\n  Version:          18.09.1\n  API version:      1.39 (minimum version 1.12)\n  Go version:       go1.10.6\n  Git commit:       4c52b90\n  Built:            Wed Jan  9 19:41:49 2019\n  OS/Arch:          linux/amd64\n  Experimental:     true\n'


In [4]:
image = backend.ImageClass("docker.io/library/nginx")

In [5]:
image.inspect()

{'Id': 'sha256:06144b2878448774e55577ae7d66b5f43a87c2e44322b3884e4e6c70d070b262',
 'RepoTags': ['nginx:latest'],
 'RepoDigests': ['nginx@sha256:24a0c4b4a4c0eb97a1aabb8e29f18e917d05abfe1b7a7c07857230879ce7d3d3'],
 'Parent': '',
 'Comment': '',
 'Created': '2018-09-05T00:57:00.322491744Z',
 'Container': 'ea0a37cdc91d1228880c77a4660fd74a6ba6ae5d2de6c54e2ac025c66c11f257',
 'ContainerConfig': {'Hostname': 'ea0a37cdc91d',
  'Domainname': '',
  'User': '',
  'AttachStdin': False,
  'AttachStdout': False,
  'AttachStderr': False,
  'ExposedPorts': {'80/tcp': {}},
  'Tty': False,
  'OpenStdin': False,
  'StdinOnce': False,
  'Env': ['PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin',
   'NGINX_VERSION=1.15.3-1~stretch',
   'NJS_VERSION=1.15.3.0.2.3-1~stretch'],
  'Cmd': ['/bin/sh', '-c', '#(nop) ', 'CMD ["nginx" "-g" "daemon off;"]'],
  'ArgsEscaped': True,
  'Image': 'sha256:674565b18e1c8e0102c657d89059818ba8e0f476073f63078439951d4e0370b0',
  'Volumes': None,
  'WorkingDir': '

In [6]:
container = image.run_via_binary()

10:33:04.605 image.py          INFO   run container via binary in background


In [7]:
container.is_running()

True

In [8]:
container.get_IPv4s()

['172.17.0.3']

In [9]:
container.get_ports()

['80']

In [10]:
container.stop()

In [11]:
container.is_running()

False

### HTTP client

In [12]:
image = backend.ImageClass("registry.fedoraproject.org/fedora")

In [15]:
container = image.run_via_binary(command=["python3", "-m", "http.server", "--bind", "0.0.0.0 8000"],
                                 additional_opts=["-p", "8000:8000"])

10:34:39.371 image.py          INFO   run container via binary in background


In [16]:
container.get_ports()

['8000']

In [17]:
container.get_IPv4s()

['172.17.0.2']

In [18]:
container.get_port_mappings()

{'8000/tcp': [{'HostIp': '0.0.0.0', 'HostPort': '8000'}]}

In [19]:
request = container.http_request(host="127.0.0.1", port="8000")

In [20]:
request.ok

True

In [21]:
display(HTML(request.content.decode("utf-8")))

In [22]:
request = container.http_request(path="/etc", host="127.0.0.1", port="8000")

In [23]:
request.ok

True

In [24]:
"passwd" in request.content.decode("utf-8")

True

# Kubernetes example

This example demonstrating how to run image created by `DockerBackend` as container inside Kubernetes pod.

In [25]:
from conu import K8sBackend

In [26]:
k8s_backend = K8sBackend()

10:35:37.207 backend.py        INFO   conu has initiated, welcome to the party!


In [27]:
docker_backend = DockerBackend()

10:35:40.844 backend.py        INFO   conu has initiated, welcome to the party!


In [28]:
image = docker_backend.ImageClass("openshift/hello-openshift")

In [29]:
pod = image.run_in_pod(namespace='myproject')

10:36:03.719 image.py          INFO   Starting Pod hello-openshift-rpitonak-prr4-pod in namespace myproject


In [30]:
pod.is_ready()

10:36:11.206 pod.py            INFO   Pod: hello-openshift-rpitonak-prr4-pod in namespace: myproject is ready!


True

In [31]:
pod.get_phase()

<PodPhase.RUNNING: 1>

In [32]:
pod.delete()

10:36:18.007 pod.py            INFO   Deleting Pod hello-openshift-rpitonak-prr4-pod in namespace myproject


In [33]:
pod.get_phase()

<PodPhase.TERMINATING: 4>

## Deployment

In [34]:
from conu.utils import get_oc_api_token

In [35]:
from conu.backend.k8s.deployment import Deployment

In [36]:
api_key = get_oc_api_token()
k8s_backend = K8sBackend(api_key=api_key)

10:37:04.401 backend.py        INFO   conu has initiated, welcome to the party!


In [37]:
template = """
        apiVersion: apps/v1
        kind: Deployment
        metadata:
          name: hello-world
          labels:
            app: hello-world
        spec:
          replicas: 3
          selector:
            matchLabels:
              app: hello-world
          template:
            metadata:
              labels:
                app: hello-world
            spec:
              containers:
              - name: hello-openshift
                image: openshift/hello-openshift
        """

In [38]:
test_deployment = Deployment(namespace='myproject', from_template=template, create_in_cluster=True)

10:37:10.667 deployment.py     INFO   Creating Deployment hello-world in namespace: myproject


In [39]:
test_deployment.all_pods_ready()

10:37:22.033 deployment.py     INFO   All pods are ready for deployment hello-world in namespace: myproject


True

In [40]:
test_deployment.get_status()

{'available_replicas': 3,
 'collision_count': None,
 'conditions': [{'last_transition_time': datetime.datetime(2019, 1, 23, 9, 37, 15, tzinfo=tzutc()),
                 'last_update_time': datetime.datetime(2019, 1, 23, 9, 37, 15, tzinfo=tzutc()),
                 'message': 'Deployment has minimum availability.',
                 'reason': 'MinimumReplicasAvailable',
                 'status': 'True',
                 'type': 'Available'},
                {'last_transition_time': datetime.datetime(2019, 1, 23, 9, 37, 9, tzinfo=tzutc()),
                 'last_update_time': datetime.datetime(2019, 1, 23, 9, 37, 15, tzinfo=tzutc()),
                 'message': 'ReplicaSet "hello-world-5d579c544d" has '
                            'successfully progressed.',
                 'reason': 'NewReplicaSetAvailable',
                 'status': 'True',
                 'type': 'Progressing'}],
 'observed_generation': 1,
 'ready_replicas': 3,
 'replicas': 3,
 'unavailable_replicas': None,
 'updat

In [41]:
test_deployment.delete()

10:38:18.481 deployment.py     INFO   Deleting Deployment hello-world in namespace: myproject


# OpenShift example

In [42]:
from conu.utils import get_oc_api_token

In [43]:
from conu import OpenshiftBackend

In [44]:
api_key = get_oc_api_token()

In [45]:
api_key

'PG-Z8nALuWCrUCj21iw7qap7DA9kokfErWEIIrZUEZY'

In [46]:
openshift_backend = OpenshiftBackend(api_key=api_key)

10:38:57.045 backend.py        INFO   conu has initiated, welcome to the party!


In [47]:
app_name = openshift_backend.deploy_image("centos/mariadb-102-centos7",
                                         oc_new_app_args=["--env", "MYSQL_ROOT_PASSWORD=test"],
                                         project='myproject')

10:39:24.919 backend.py        INFO   Importing image from: centos/mariadb-102-centos7, as: mariadb-102-centos7
10:39:27.096 backend.py        INFO   Creating new app in project myproject


In [48]:
openshift_backend.all_pods_are_ready(app_name)

10:39:36.686 pod.py            INFO   Pod: app-czwxd-1-qjnqn in namespace: myproject is ready!
10:39:36.687 backend.py        INFO   All pods are ready!


True

In [49]:
logs = openshift_backend.get_logs(app_name)
display(Pretty(logs))

In project My Project (myproject) on server https://192.168.99.100:8443

svc/app-czwxd - 172.30.118.101:3306
  dc/app-czwxd deploys istag/mariadb-102-centos7:latest 
    deployment #1 deployed 20 seconds ago - 1 pod


2 infos identified, use 'oc status -v' to see details.
=> sourcing 20-validate-variables.sh ...
=> sourcing 25-validate-replication-variables.sh ...
=> sourcing 30-base-config.sh ...
---> 09:39:30     Processing basic MySQL configuration files ...
=> sourcing 60-replication-config.sh ...
=> sourcing 70-s2i-config.sh ...
---> 09:39:30     Processing additional arbitrary  MySQL configuration provided by s2i ...
=> sourcing 40-paas.cnf ...
=> sourcing 50-my-tuning.cnf ...
---> 09:39:30     Initializing database ...
---> 09:39:30     Running mysql_install_db ...
2019-01-23  9:39:30 140016118802624 [Note] Using unique option prefix 'ignore-db-dir' is error-prone and can break in the future. Please use the full name 'ignore_db_dirs' instead.
2019-01-23  9:39:30 140016118802624 

In [50]:
openshift_backend.clean_project(app_name)

10:41:07.530 backend.py        INFO   Deleting all objects with label app=app-czwxd
10:41:10.126 backend.py        INFO   pod "app-czwxd-1-qjnqn" deleted
10:41:10.128 backend.py        INFO   replicationcontroller "app-czwxd-1" deleted
10:41:10.131 backend.py        INFO   service "app-czwxd" deleted
10:41:10.132 backend.py        INFO   deploymentconfig.apps.openshift.io "app-czwxd" deleted
10:41:10.133 backend.py        INFO   


## Creating application from source

In [51]:
status = openshift_backend.get_status()
display(Pretty(status))

In project My Project (myproject) on server https://192.168.99.100:8443

You have no services, deployment configs, or build configs.
Run 'oc new-app' to create an application.


In [52]:
app_name = openshift_backend.create_new_app_from_source("centos/python-36-centos7",
                                                       source="https://github.com/openshift/django-ex.git",
                                                       project="myproject")

10:42:05.192 backend.py        INFO   Importing image from: centos/python-36-centos7, as: python-36-centos7
10:42:06.592 backend.py        INFO   Creating new app in project myproject


In [53]:
status = openshift_backend.get_status()
display(Pretty(status))

In project My Project (myproject) on server https://192.168.99.100:8443

svc/app-bdxnk - 172.30.155.244:8080
  dc/app-bdxnk deploys istag/app-bdxnk:latest <-
    bc/app-bdxnk source builds https://github.com/openshift/django-ex.git on istag/python-36-centos7:latest 
      build #1 running for 8 seconds - ab765c5: Merge pull request #134 from liangxia/okd (Honza Horak <hhorak@redhat.com>)
    deployment #1 waiting on image or update


2 infos identified, use 'oc status -v' to see details.


In [55]:
openshift_backend.all_pods_are_ready(app_name)

10:42:34.322 pod.py            INFO   Pod: app-bdxnk-1-sfdkf in namespace: myproject is ready!
10:42:34.323 backend.py        INFO   All pods are ready!


True

In [59]:
openshift_backend.clean_project(app_name)

10:52:16.055 backend.py        INFO   Deleting all objects with label app=app-bdxnk
10:52:18.673 backend.py        INFO   pod "app-bdxnk-1-sfdkf" deleted
10:52:18.675 backend.py        INFO   replicationcontroller "app-bdxnk-1" deleted
10:52:18.676 backend.py        INFO   service "app-bdxnk" deleted
10:52:18.677 backend.py        INFO   deploymentconfig.apps.openshift.io "app-bdxnk" deleted
10:52:18.678 backend.py        INFO   buildconfig.build.openshift.io "app-bdxnk" deleted
10:52:18.678 backend.py        INFO   build.build.openshift.io "app-bdxnk-1" deleted
10:52:18.679 backend.py        INFO   imagestream.image.openshift.io "app-bdxnk" deleted
10:52:18.680 backend.py        INFO   


For more information see upstream GitHub repository: https://github.com/user-cont/conu. You can find there more examples for all supported backends.